# Anomaly Detection for UNSW NB-15 cyberattack dataset
Identifying cyberattacks can be considered both a classification problem and an anomaly detection problem. In this notebook, I treat it as an anomaly detection problem. [The training and test csv's provided by the researchers](https://cloudstor.aarnet.edu.au/plus/index.php/s/2DhnLGDdEECo4ys?path=%2FUNSW-NB15%20-%20CSV%20Files%2Fa%20part%20of%20training%20and%20testing%20set) are balanced between the normal and anomaly classes, to the extent that I'd like more normal traffic examples on which to train my outlier detection models.

In [1]:
# Custom modules
import data_prep as dp 
import model_abstraction as moda

# Data Structures
import pandas as pd
import numpy as np
import pickle as pkl

# Preprocessing or data manipulation methods
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Modeling methods and selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, StratifiedKFold

# Anomaly detection
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor

# Model assessment
from sklearn.metrics import confusion_matrix, roc_auc_score

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Modeling with more developed features
The model selection process is similar to that of the previous notebook, however in this attempt the dataset is different. I've added more contextual features as one-hot-encoded variables, I've removed some features that were not useful in differentiating between attack and normal data, and have incorporated standard scaling for the numeric features that remain. It won't mitigate the curse of dimensionality entirely, but the data points in this set should be "closer" together than in the unscaled instance.

In [3]:
# Load train and test data
X_train, y_train = dp.load_agg_Xy(path='./data/', sample_size=0.1, strat_cat='label')

# Reformat for anomaly detection labeling
y_train = y_train.apply(dp.y_anomaly_format)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3265: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3265: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# Create holdout set to verify generalizability of results
X_train, X_hold, y_train, y_hold  = train_test_split(X_train,y_train, test_size = 0.25,
                                                     random_state = 42, stratify=y_train)

In [5]:
# How much data are we working with?
X_train.shape, X_hold.shape

((190503, 39), (63502, 39))

## Methodology for Scikit-Learn anomaly detection algorithms
The type of anomaly detection methods used below start by training on normal data and effectively building a profile as to what normal instances should look like. From there, they can be used to predict on unseen data based on the previously established boundary.

In [6]:
## Create 'masks' to filter the dataframe by whether or not the observation belongs to normal or attack class.
## Use this to isolate only normal training data for fitting the models.
train_normal = y_train==1

In [7]:
## Bring in column transformations to process contextual features and standardize numerical values
#with open('ct_ohe_ssc_xyagg.pkl', 'rb') as f:
#    col_trans = pkl.load(f)

# Define the individual steps
ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
ssc_step = ('std_sclr', StandardScaler())

# Make the step part of a pipeline
ohe_pipe = Pipeline([ohe_step])
ssc_pipe = Pipeline([ssc_step])

# Columns to transform: categorical columns for encoding, numeric feature columns for standardizing
ohe_cols = ['proto', 'state', 'service']
binary_cols = ['is_sm_ips_ports', 'is_ftp_login']
non_ssc_cols = ohe_cols+binary_cols
ssc_cols = [col for col in X_train.columns if col not in non_ssc_cols]

# Transformer input: tuple w/ contents ('name', SomeTransformer(Parameters), columns)
transformer = [
    ('one_hot_encoding', ohe_pipe, ohe_cols),
    ('standard_scaling', ssc_pipe, ssc_cols)
]
col_trans = ColumnTransformer(transformers=transformer, remainder='passthrough')

## Isolation Forest
Isolation forests operate similarly in principle to the random forest classifier in that they use information gain, or information entropy, to separate data based on a few variables. Compared to the other algorithms showcased below, it trains relatively quickly.

In [8]:
ilf = IsolationForest(random_state=0)
ilf.fit(col_trans.fit_transform(X_train[train_normal]))


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:214: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:224: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


IsolationForest(behaviour='old', bootstrap=False, contamination='legacy',
        max_features=1.0, max_samples='auto', n_estimators=100,
        n_jobs=None, random_state=0, verbose=0)

The cells below look at isolation forest performance on the training and holdout sets to get a performance estimate.

In [9]:
y_if_train = ilf.predict(col_trans.transform(X_train))
y_if_hold = ilf.predict(col_trans.transform(X_hold))

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:418: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:418: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.

In [10]:
# The true outliers are in the top row and the true inliers in the second.
# We want the diagonal values to be as high as possible.
confusion_matrix(y_train, y_if_train)

array([[ 23666,    430],
       [ 16641, 149766]])

In [11]:
print(roc_auc_score(y_hold, y_if_hold))
confusion_matrix(y_hold, y_if_hold)

0.9387241079848625


array([[ 7861,   171],
       [ 5617, 49853]])

## Basic One Class SVM

In [ ]:
ocsvm = OneClassSVM(kernel='rbf')
ocsvm.fit(col_trans.transform(X_train[train_normal]))
y_ocsvm_nm = ocsvm.fit_predict(col_trans.transform(X_train[train_normal]))
y_ocsvm_out = ocsvm.predict(col_trans.transform(X_train[~train_normal]))
y_ocsvm_train = ocsvm.predict(col_trans.transform(X_train))
y_ocsvm_hold = ocsvm.predict(col_trans.transform(X_hold))

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", F

In [ ]:
confusion_matrix(y_train[train_normal], y_ocsvm_nm)

In [ ]:
confusion_matrix(y_train[~train_normal], y_ocsvm_out)

In [ ]:
confusion_matrix(y_train, y_ocsvm_train)

In [ ]:
roc_auc_score(y_hold, y_ocsvm_hold)

## Eliptical Envelope

In [12]:
eenv = EllipticEnvelope(contamination=0.2, random_state=0)
eenv.fit(col_trans.transform(X_train[train_normal]))

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:646: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


EllipticEnvelope(assume_centered=False, contamination=0.2, random_state=0,
         store_precision=True, support_fraction=None)

In [14]:
roc_auc_score(y_hold,eenv.predict(col_trans.transform(X_hold)))

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.6831102083463514

# Local Outlier Factor

In [59]:
lof = LocalOutlierFactor()
y_pred = lof.fit_predict(X_train)

/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


In [ ]:
y_pred

In [43]:
neg_scores = lof.negative_outlier_factor_
out_scores = (neg_scores.max()-neg_scores)/(neg_scores.max()-neg_scores.min())

In [53]:
pd.Series(neg_scores).describe()

count    1.315050e+05
mean    -7.703878e+12
std      2.633116e+14
min     -2.320057e+16
25%     -1.078605e+00
50%     -1.004495e+00
75%     -1.000000e+00
max     -8.769621e-01
dtype: float64

In [54]:
for lim in np.linspace(1e-5,1e1,10):
    print(roc_auc_score(y_train, list(map(y_anomaly_format, out_scores > lim))))

0.5005816494689044
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5


In [61]:
roc_auc_score(y_train, y_pred)

0.5393576376419513